# Оксана Рудковская, коллаборативная фильтрация

ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже
Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [61]:
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

### Датасет на 1М

In [45]:
movies = pd.read_csv('ml-latest\movies.csv')
ratings = pd.read_csv(r'ml-latest\ratings.csv')

In [47]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [49]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [50]:
dataset.head()

,uid,iid,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0


In [53]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [58]:
trainset, testset = train_test_split(data, test_size=.60)

In [25]:
from surprise.model_selection import KFold
from surprise import SVD

In [60]:
kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.7950
RMSE: 0.7941
RMSE: 0.7945
RMSE: 0.7944
RMSE: 0.7945


### Датасет на на 100K

In [62]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV


In [63]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [64]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [65]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [66]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [69]:
algo = SVD(n_epochs= 50, lr_all= 0.015, reg_all=0.2, init_std_dev=0.2)
cross_validate(algo, data, cv=5, verbose=True,measures=[u'rmse'])

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8695  0.8712  0.8604  0.8617  0.8694  0.8664  0.0044  
Fit time          11.17   11.09   11.30   11.70   11.47   11.35   0.22    
Test time         0.17    0.12    0.11    0.12    0.11    0.13    0.02    


{'test_rmse': array([0.86946452, 0.8711745 , 0.86041431, 0.86174065, 0.86935872]),
 'fit_time': (11.169565677642822,
  11.087415218353271,
  11.302331924438477,
  11.703609704971313,
  11.472856998443604),
 'test_time': (0.17171430587768555,
  0.12092304229736328,
  0.11269474029541016,
  0.11982059478759766,
  0.10571765899658203)}